In [1]:
#数据处理部分之前的代码，加入部分数据处理的库
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import os
import gzip
import json
import random
import numpy as np

In [2]:
# 声明数据集文件位置
datafile = './data/mnist.json.gz'
print('loading mnist dataset from {} ......'.format(datafile))
# 加载json数据文件
data = json.load(gzip.open(datafile))
print('mnist dataset load done')

loading mnist dataset from ./data/mnist.json.gz ......
mnist dataset load done


In [3]:
train_set, val_set, eval_set = data

In [8]:
len(train_set[0])

50000

In [16]:
img1 = train_set[0][0]
len(img1)
type(img1)

list

In [9]:
# 观察训练集数据
imgs, labels = train_set[0], train_set[1]
print("训练数据集数量: ", len(imgs))

# 观察验证集数量
imgs, labels = val_set[0], val_set[1]
print("验证数据集数量: ", len(imgs))

# 观察测试集数量
imgs, labels = val= eval_set[0], eval_set[1]
print("测试数据集数量: ", len(imgs))

训练数据集数量:  50000
验证数据集数量:  10000
测试数据集数量:  10000


In [10]:
imgs, labels = train_set[0], train_set[1]
print("训练数据集数量: ", len(imgs))
# 获得数据集长度
imgs_length = len(imgs)
# 定义数据集每个数据的序号，根据序号读取数据
index_list = list(range(imgs_length))
# 读入数据时用到的批次大小
BATCHSIZE = 100

# 随机打乱训练数据的索引序号
random.shuffle(index_list)

训练数据集数量:  50000


In [11]:
def data_generator():
    imgs_list = []
    labels_list = []
    for i in index_list:
        # 将数据处理成希望的类型
        img = np.array(imgs[i]).astype('float32')
        label = np.array(labels[i]).astype('float32')
        imgs_list.append(img) 
        labels_list.append(label)
        if len(imgs_list) == BATCHSIZE:
            # 获得一个batchsize的数据，并返回
            yield np.array(imgs_list), np.array(labels_list)
            # 清空数据读取列表
            imgs_list = []
            labels_list = []

    # 如果剩余数据的数目小于BATCHSIZE，
    # 则剩余数据一起构成一个大小为len(imgs_list)的mini-batch
    if len(imgs_list) > 0:
        yield np.array(imgs_list), np.array(labels_list)
    return data_generator

In [12]:
# 声明数据读取函数，从训练集中读取数据
train_loader = data_generator
# 以迭代的形式读取数据
for batch_id, data in enumerate(train_loader()):
    image_data, label_data = data
    if batch_id == 0:
        # 打印数据shape和类型
        print("打印第一个batch数据的维度:")
        print("图像维度: {}, 标签维度: {}".format(image_data.shape, label_data.shape))
    break

打印第一个batch数据的维度:
图像维度: (100, 784), 标签维度: (100,)


In [17]:
import numpy as np
from paddle.io import Dataset
# 构建一个类，继承paddle.io.Dataset，创建数据读取器
class RandomDataset(Dataset):
    def __init__(self, num_samples):
        # 样本数量
        self.num_samples = num_samples

    def __getitem__(self, idx):
        # 随机产生数据和label
        image = np.random.random([784]).astype('float32')
        label = np.random.randint(0, 9, (1, )).astype('float32')
        return image, label

    def __len__(self):
        # 返回样本总数量
        return self.num_samples
        
# 测试数据读取器
dataset = RandomDataset(10)
for i in range(len(dataset)):
    print(dataset[i])

(array([0.31726444, 0.80038434, 0.3135445 , 0.01317841, 0.16873637,
       0.641604  , 0.35156217, 0.89608335, 0.6176258 , 0.47519916,
       0.24513003, 0.5739105 , 0.3577325 , 0.9047216 , 0.1634517 ,
       0.21508364, 0.06571486, 0.06082624, 0.00941431, 0.1281093 ,
       0.8949699 , 0.16427939, 0.85276747, 0.06315114, 0.8246493 ,
       0.09651058, 0.770558  , 0.6036645 , 0.4820524 , 0.405835  ,
       0.997545  , 0.91473305, 0.89498687, 0.41108793, 0.4477046 ,
       0.49182883, 0.8162353 , 0.17769842, 0.7854691 , 0.2157255 ,
       0.2744975 , 0.08683734, 0.82696164, 0.5691687 , 0.09573324,
       0.2815677 , 0.9445864 , 0.28930128, 0.8849621 , 0.59054095,
       0.7542988 , 0.8913719 , 0.72765714, 0.9260076 , 0.27252582,
       0.05473283, 0.54843014, 0.09420645, 0.67373323, 0.13591851,
       0.41720432, 0.1059849 , 0.81598485, 0.1192643 , 0.12461247,
       0.18174328, 0.38578787, 0.74322546, 0.30384344, 0.30915555,
       0.4121824 , 0.39957678, 0.14787416, 0.1908261 , 0.0260

In [18]:
loader = paddle.io.DataLoader(dataset, batch_size=3, shuffle=True, drop_last=True, num_workers=2)
for i, data in enumerate(loader()):
    images, labels = data[0], data[1]
    print("batch_id: {}, 训练数据shape: {}, 标签数据shape: {}".format(i, images.shape, labels.shape))

d:\Anaconda\envs\pytorch\lib\site-packages\paddle\fluid\reader.py:486: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  warnings.warn(


batch_id: 0, 训练数据shape: [3, 784], 标签数据shape: [3, 1]
batch_id: 1, 训练数据shape: [3, 784], 标签数据shape: [3, 1]
batch_id: 2, 训练数据shape: [3, 784], 标签数据shape: [3, 1]


In [ ]:
class MnistDataset(paddle.io.Dataset):
    def __init__(self, mode):
        datafile = './work/mnist.json.gz'
        data = json.load(gzip.open(datafile))
        # 读取到的数据区分训练集，验证集，测试集
        train_set, val_set, eval_set = data
        if mode=='train':
            # 获得训练数据集
            imgs, labels = train_set[0], train_set[1]
        elif mode=='valid':
            # 获得验证数据集
            imgs, labels = val_set[0], val_set[1]
        elif mode=='eval':
            # 获得测试数据集
            imgs, labels = eval_set[0], eval_set[1]
        else:
            raise Exception("mode can only be one of ['train', 'valid', 'eval']")
        
        # 校验数据
        imgs_length = len(imgs)
        assert len(imgs) == len(labels), \
            "length of train_imgs({}) should be the same as train_labels({})".format(len(imgs), len(labels))
        
        self.imgs = imgs
        self.labels = labels

    def __getitem__(self, idx):
        img = np.array(self.imgs[idx]).astype('float32')
        label = np.array(self.labels[idx]).astype('float32')
        
        return img, label

    def __len__(self):
        return len(self.imgs)
    

# 声明数据加载函数，使用MnistDataset数据集
train_dataset = MnistDataset(mode='train')
# 使用paddle.io.DataLoader 定义DataLoader对象用于加载Python生成器产生的数据，
# DataLoader 返回的是一个批次数据迭代器，并且是异步的；
data_loader = paddle.io.DataLoader(train_dataset, batch_size=100, shuffle=True)
# 迭代的读取数据并打印数据的形状
for i, data in enumerate(data_loader()):
    images, labels = data
    